# Gesamtprogramm

In [44]:
import ImageHandler as ih
import numpy as np
import pandas as pd
import random
from ImageHandler import cv
from pprint import pprint

loader = ih.ImageLoader('bilddaten\Dosenoeffner')
loader.AddDirRealPath('bilddaten\Flaschenoeffner')
loader.AddDirRealPath('bilddaten\Korkenzieher')

# loader.AddDirRealPath('train\Dosenoeffner')
# loader.AddDirRealPath('train\Flaschenoeffner')
# loader.AddDirRealPath('train\Korkenzieher')

# loader.AddDirRealPath('iliasdaten\Dosenoeffner')
# loader.AddDirRealPath('iliasdaten\Flaschenoeffner')
# loader.AddDirRealPath('iliasdaten\Korkenzieher')

Added Directory bilddaten\Dosenoeffner
Added Directory bilddaten\Flaschenoeffner
Added Directory bilddaten\Korkenzieher


In [45]:
loader.MakeSampleEntrys(3)

print('dateilänge = ',len(loader.allImgPaths))
print('Randomlänge = ',len(loader.sampleEntrysPath))

changer = ih.ImageChanger(loader.sampleEntrysPath, 0, False)

dateilänge =  3068
Randomlänge =  3


In [46]:

changer.FrameByWidth(400)
changer.BilateralFilter(10,10)
# changer.ShowAll()

changer.Normieren()
changer.ColorAdapt(sigma=1)
# changer.ShowAll()

changer.BrightTresh()
# changer.ShowAll()

changer.Bordering(100)
# changer.ShowAll()

changer.InvertCheck()
# changer.ShowAll()

changer.Dilation(7,2)
# changer.ShowAll()

changer.Erode(7,2)
# changer.ShowAll()

changer.ConturMerkmale(type1=cv.RETR_TREE,type2=cv.CHAIN_APPROX_NONE,epsyValue=0.005)
# changer.ShowAll()
changer.PrintErrors(all=True)
changer.PrintValuesToCSV('extracsvAll1.csv')

Total Number of Errors = 0
-------------Values CSV Done------------


In [47]:
df = pd.read_csv("extracsvAll.csv")

label_name = "type"
df = df.rename(columns={label_name:'label'})
df.head()

df_tree = df
df_bay = df

In [48]:

#df_tree = df_tree.drop("approxAnzahl",axis=1)
#df_tree = df_tree.drop("areaRatio",axis=1)
df_tree = df_tree.drop("circleLike",axis=1)
#df_tree = df_tree.drop("contureDistance",axis=1)
#df_tree = df_tree.drop("conturenAnzahl",axis=1)
#df_tree = df_tree.drop("farPoint",axis=1)
#df_tree = df_tree.drop("keyPointsAnzahl",axis=1)
df_tree = df_tree.drop("mean_val",axis=1)
df_tree = df_tree.drop("middlepointDistance",axis=1)
#df_tree = df_tree.drop("minimal_ratio",axis=1)
df_tree = df_tree.drop("minimal_ratio2",axis=1)
#df_tree = df_tree.drop("momentpointDistance",axis=1)
df_tree = df_tree.drop("rectangleLike",axis=1)
#df_tree = df_tree.drop("seradity",axis=1)
df_tree = df_tree.drop("seradity2",axis=1)
#df_tree = df_tree.drop("solidity",axis=1)
df_tree = df_tree.drop("solidity2",axis=1)
#df_tree = df_tree.drop("steiner",axis=1)
df_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   approxAnzahl         752 non-null    int64  
 1   areaRatio            752 non-null    float64
 2   contureDistance      752 non-null    float64
 3   conturenAnzahl       752 non-null    int64  
 4   farPoint             752 non-null    int64  
 5   keyPointsAnzahl      752 non-null    int64  
 6   minimal_ratio        752 non-null    float64
 7   momentpointDistance  752 non-null    float64
 8   seradity             752 non-null    float64
 9   solidity             752 non-null    float64
 10  steiner              752 non-null    float64
 11  label                752 non-null    object 
dtypes: float64(7), int64(4), object(1)
memory usage: 70.6+ KB


In [49]:
dec = ih.Dectree()
train_df, test_df = dec.frame_split(df_tree, test_size_in_percent=20)

In [50]:
tree = dec.make_decision_tree(train_df,min_samples=1, max_depth=15)
print(tree)

{'seradity <= 0.8487558929054093': [{'farPoint <= 44219.0': [{'minimal_ratio <= 2.353810040839819': [{'solidity <= 0.7068442050650401': [{'solidity <= 0.5330628546718854': [{'seradity <= 0.5554176630791391': ['Flaschenoeffner', {'steiner <= 22.416442414621063': [{'steiner <= 19.561867194619055': ['Korkenzieher', 'Flaschenoeffner']}, 'Korkenzieher']}]}, {'momentpointDistance <= 17.5': [{'seradity <= 0.7770744926835151': [{'solidity <= 0.5898522019041643': ['Flaschenoeffner', 'Dosenoeffner']}, 'Korkenzieher']}, {'contureDistance <= 1.0609921418872994': [{'minimal_ratio <= 1.7012815813422204': ['Flaschenoeffner', {'steiner <= 70.8094344916361': ['Korkenzieher', 'Dosenoeffner']}]}, 'Korkenzieher']}]}]}, {'approxAnzahl <= 23.5': ['Dosenoeffner', {'contureDistance <= 1.1213588365317055': [{'momentpointDistance <= 13.564822349438947': ['Flaschenoeffner', 'Korkenzieher']}, {'steiner <= 9.44396434292553': ['Dosenoeffner', 'Flaschenoeffner']}]}]}]}, {'momentpointDistance <= 64.98034157205745': [

In [51]:
accuracy, keynumbers = dec.calculate_keynumbers(test_df, tree)
accuracy

0.9266666666666666

In [52]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers

,Label,recall,precision,f1score
0,Dosenoeffner,0.878788,0.950820,0.913386
1,Flaschenoeffner,0.795918,0.975000,0.876404
2,Korkenzieher,0.913043,0.857143,0.884211


# Bayes

In [53]:
#df_bay = df_bay.drop("approxAnzahl",axis=1)
#df_bay = df_bay.drop("areaRatio",axis=1)
df_bay = df_bay.drop("circleLike",axis=1)
#df_bay = df_bay.drop("contureDistance",axis=1)
#df_bay = df_bay.drop("conturenAnzahl",axis=1)
#df_bay = df_bay.drop("farPoint",axis=1)
#df_bay = df_bay.drop("keyPointsAnzahl",axis=1)
df_bay = df_bay.drop("mean_val",axis=1)
df_bay = df_bay.drop("middlepointDistance",axis=1)
#df_bay = df_bay.drop("minimal_ratio",axis=1)
df_bay = df_bay.drop("minimal_ratio2",axis=1)
#df_bay = df_bay.drop("momentpointDistance",axis=1)
df_bay = df_bay.drop("rectangleLike",axis=1)
#df_bay = df_bay.drop("seradity",axis=1)
df_bay = df_bay.drop("seradity2",axis=1)
#df_bay = df_bay.drop("solidity",axis=1)
df_bay = df_bay.drop("solidity2",axis=1)
#df_bay = df_bay.drop("steiner",axis=1)
df_bay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   approxAnzahl         752 non-null    int64  
 1   areaRatio            752 non-null    float64
 2   contureDistance      752 non-null    float64
 3   conturenAnzahl       752 non-null    int64  
 4   farPoint             752 non-null    int64  
 5   keyPointsAnzahl      752 non-null    int64  
 6   minimal_ratio        752 non-null    float64
 7   momentpointDistance  752 non-null    float64
 8   seradity             752 non-null    float64
 9   solidity             752 non-null    float64
 10  steiner              752 non-null    float64
 11  label                752 non-null    object 
dtypes: float64(7), int64(4), object(1)
memory usage: 70.6+ KB


In [54]:
bay = ih.Bayes()

df_bayes = df_bay.round(1)
df_train, df_test = bay.frame_split(df_bayes, test_size_in_percent=20)

test_labels = df_test["label"]
df_test = df_test.drop("label",axis=1)
df_test

,approxAnzahl,areaRatio,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,minimal_ratio,momentpointDistance,seradity,solidity,steiner
371,19,0.1,0.9,3,7955,415,4.5,35.5,0.9,0.8,18.4
345,20,0.0,1.5,11,12695,427,2.2,29.6,0.8,0.7,24.8
427,19,0.1,0.9,3,7955,415,4.5,35.5,0.9,0.8,18.4
409,19,0.5,0.9,16,19058,462,2.6,5.0,0.8,0.6,27.5
190,13,1.0,1.1,1,5169,242,3.1,52.0,1.0,0.9,9.8
...,...,...,...,...,...,...,...,...,...,...,...
328,33,0.0,1.1,8,20971,500,1.8,66.1,0.7,0.7,4.1
166,22,0.0,0.9,2,55529,426,2.2,-8.1,0.6,0.6,24.3
335,14,0.2,1.1,3,4075,438,4.8,23.0,1.0,0.8,3.2
142,4,0.4,0.6,4,0,272,1.3,237.0,1.0,1.0,1.0


In [55]:
df_train

,approxAnzahl,areaRatio,contureDistance,conturenAnzahl,farPoint,keyPointsAnzahl,minimal_ratio,momentpointDistance,seradity,solidity,steiner,label
0,26,0.0,1.0,3,65806,434,2.7,-8.2,0.6,0.7,5.1,Dosenoeffner
1,27,0.1,1.1,2,29660,432,2.0,24.3,0.7,0.5,30.2,Korkenzieher
2,20,0.1,1.3,2,7145,418,3.0,23.8,0.9,0.7,16.3,Flaschenoeffner
5,18,0.1,1.1,2,7794,310,3.7,29.7,0.9,0.8,10.6,Flaschenoeffner
6,17,0.2,1.1,2,8871,422,2.9,25.8,0.9,0.8,18.0,Dosenoeffner
...,...,...,...,...,...,...,...,...,...,...,...,...
747,23,1.0,1.0,1,8976,353,2.5,50.2,0.9,0.8,5.8,Dosenoeffner
748,21,0.0,1.3,2,5833,423,3.4,33.2,0.9,0.8,5.0,Flaschenoeffner
749,13,0.0,1.0,2,3617,346,5.6,19.7,0.9,0.8,20.0,Flaschenoeffner
750,14,0.0,1.3,2,1595,369,3.1,51.5,0.9,1.0,1.4,Flaschenoeffner


In [56]:
bayes_table = bay.wahrscheinlichkeiten(df_train, label_column="label")
pprint(bayes_table)

{'approxAnzahl': {4: array([0.01244813, 0.08189655, 0.03508772]),
                  6: array([0.00414938, 0.00862069, 0.00438596]),
                  8: array([0.00414938, 0.02586207, 0.00438596]),
                  9: array([0.00414938, 0.01293103, 0.00438596]),
                  11: array([0.01659751, 0.03017241, 0.00438596]),
                  12: array([0.02074689, 0.07758621, 0.00438596]),
                  13: array([0.03319502, 0.03017241, 0.00877193]),
                  14: array([0.00414938, 0.06465517, 0.00877193]),
                  15: array([0.04149378, 0.04741379, 0.01754386]),
                  16: array([0.0373444 , 0.06465517, 0.01754386]),
                  17: array([0.05809129, 0.05172414, 0.00438596]),
                  18: array([0.09958506, 0.06896552, 0.01754386]),
                  19: array([0.05809129, 0.03448276, 0.02631579]),
                  20: array([0.09958506, 0.04310345, 0.04385965]),
                  21: array([0.09543568, 0.0387931 , 0.03947368]),

In [57]:
accuracy, keynumbers = bay.calculate_keynumbers(test_df, bayes_table)
accuracy

0.9066666666666666

In [58]:
keynumbers = pd.DataFrame(keynumbers)
keynumbers

,Label,recall,precision,f1score
0,Dosenoeffner,0.840580,0.950820,0.892308
1,Flaschenoeffner,0.770833,0.925000,0.840909
2,Korkenzieher,0.872340,0.836735,0.854167
